In [1]:
import pandas as pd

In [2]:
matches2324 = pd.read_csv("C:/Users/tomas/source/repos/Footy/data/matches.csv", index_col=0)

In [3]:
matches2324.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2.0,1.0,Nott'ham Forest,...,Match Report,NaN,15.0,7.0,19.1,0.0,0,0,2024,Arsenal
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,Match Report,NaN,13.0,2.0,16.4,0.0,1,1,2024,Arsenal
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,Match Report,NaN,18.0,9.0,13.8,0.0,1,1,2024,Arsenal
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3.0,1.0,Manchester Utd,...,Match Report,NaN,17.0,5.0,15.0,0.0,0,0,2024,Arsenal
5,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1.0,0.0,Everton,...,Match Report,NaN,13.0,4.0,17.4,0.0,0,0,2024,Arsenal


In [4]:
matches2324.shape

(2234, 27)

In [5]:
matches2324.columns

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'referee', 'match report', 'notes', 'sh', 'sot', 'dist', 'fk', 'pk',
       'pkatt', 'season', 'team'],
      dtype='object')

In [6]:
matches2025 = pd.read_csv("C:/Users/tomas/source/repos/Footy/data/matches2025.csv", index_col=0)


In [7]:
matches2025.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2024-08-18,16:30,Premier League,Matchweek 1,Sun,Away,W,2.0,0.0,Chelsea,...,Match Report,NaN,11,5,19.1,0.0,0,0,2025,ManchesterCity
2,2024-08-24,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,1.0,Ipswich Town,...,Match Report,NaN,13,4,17.8,1.0,1,1,2025,ManchesterCity
3,2024-08-31,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,1.0,West Ham,...,Match Report,NaN,23,8,15.0,1.0,0,0,2025,ManchesterCity
4,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Brentford,...,Match Report,NaN,18,7,17.2,0.0,0,0,2025,ManchesterCity
6,2024-09-22,16:30,Premier League,Matchweek 5,Sun,Home,D,2.0,2.0,Arsenal,...,Match Report,NaN,33,11,20.2,1.0,0,0,2025,ManchesterCity


In [8]:
matches2025 = matches2025.drop("opp formation", axis=1)

In [9]:
matches2025.columns

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'referee', 'match report', 'notes', 'sh', 'sot', 'dist', 'fk', 'pk',
       'pkatt', 'season', 'team'],
      dtype='object')

In [ ]:
standings_url = 'https://fbref.com/en/comps/9/Premier-League-Stats'

In [ ]:
all_matches = []

In [ ]:
import time
import requests
from bs4 import BeautifulSoup


data = requests.get(standings_url)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0] #css selector to select table with class name stats_table

links = [l.get('href') for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]  #only want the links with /squads/ in it
team_urls = [f"https://fbref.com{l}" for l in links] #create list of all the teams urls/links


for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", "")

    data = requests.get(team_url)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
    time.sleep(5)

    soup = BeautifulSoup(data.text)
    links = [l.get('href') for l in soup.find_all("a")]
    links = [l for l in links if l and 'all_comps/shooting/' in l] #get link to shooting stats for team 
    link = f"https://fbref.com{links[0]}"
    data = requests.get(link)
    shooting = pd.read_html(data.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()

    try:
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
    except ValueError:
        continue
    
    team_data = matches[matches["Comp"] == "Premier League"]
    print(team_name)
    team_data["Season"] = 2025 #adding season column
    team_data["Team"] = team_name #adding team collumn
    all_matches.append(team_data)
    time.sleep(1)

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df = match_df[match_df["round"] == "Matchweek 11"]
match_df = match_df[match_df["venue"] == "Home"]

In [ ]:
match_df = match_df.reset_index()
match_df = match_df.drop("index", axis = 1)

In [10]:
matches = pd.concat([matches2324, matches2025])
matches = matches.reset_index()
matches = matches.drop("index", axis=1)
matches

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2.0,1.0,Nott'ham Forest,...,Match Report,NaN,15.0,7.0,19.1,0.0,0,0,2024,Arsenal
1,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,Match Report,NaN,13.0,2.0,16.4,0.0,1,1,2024,Arsenal
2,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,Match Report,NaN,18.0,9.0,13.8,0.0,1,1,2024,Arsenal
3,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3.0,1.0,Manchester Utd,...,Match Report,NaN,17.0,5.0,15.0,0.0,0,0,2024,Arsenal
4,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1.0,0.0,Everton,...,Match Report,NaN,13.0,4.0,17.4,0.0,0,0,2024,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2409,2024-09-21,15:00,Premier League,Matchweek 5,Sat,Home,D,1.0,1.0,Ipswich Town,...,Match Report,NaN,11.0,3.0,13.7,0.0,0,0,2025,Southampton
2410,2024-09-30,20:00,Premier League,Matchweek 6,Mon,Away,L,1.0,3.0,Bournemouth,...,Match Report,NaN,9.0,3.0,18.4,0.0,0,0,2025,Southampton
2411,2024-10-05,15:00,Premier League,Matchweek 7,Sat,Away,L,1.0,3.0,Arsenal,...,Match Report,NaN,8.0,2.0,16.4,0.0,0,0,2025,Southampton
2412,2024-10-19,15:00,Premier League,Matchweek 8,Sat,Home,L,2.0,3.0,Leicester City,...,Match Report,NaN,14.0,7.0,9.6,0.0,0,0,2025,Southampton


In [11]:
matches.shape

(2414, 27)

In [12]:
matches2324.shape

(2234, 27)

In [13]:
matches["team"].value_counts()

team
Arsenal                   121
Liverpool                 121
WestHamUnited             121
AstonVilla                121
CrystalPalace             121
WolverhamptonWanderers    121
Everton                   121
Brentford                 121
ManchesterCity            120
TottenhamHotspur          120
BrightonandHoveAlbion     120
ManchesterUnited          120
Chelsea                   120
NewcastleUnited           120
Southampton                85
LeicesterCity              85
Bournemouth                83
Fulham                     83
NottinghamForest           83
LeedsUnited                76
Burnley                    74
NorwichCity                38
Watford                    38
LutonTown                  36
SheffieldUnited            36
IpswichTown                 9
Name: count, dtype: int64

In [14]:
matches["round"].value_counts()

round
Matchweek 1     80
Matchweek 2     80
Matchweek 3     80
Matchweek 4     80
Matchweek 5     80
Matchweek 6     80
Matchweek 7     80
Matchweek 8     80
Matchweek 9     80
Matchweek 10    60
Matchweek 11    60
Matchweek 12    60
Matchweek 13    60
Matchweek 14    60
Matchweek 15    60
Matchweek 16    60
Matchweek 17    60
Matchweek 18    60
Matchweek 19    60
Matchweek 20    60
Matchweek 21    60
Matchweek 22    60
Matchweek 23    60
Matchweek 24    60
Matchweek 25    60
Matchweek 26    60
Matchweek 27    60
Matchweek 28    60
Matchweek 30    60
Matchweek 31    60
Matchweek 32    60
Matchweek 33    60
Matchweek 29    60
Matchweek 35    60
Matchweek 36    60
Matchweek 34    54
Matchweek 37    40
Matchweek 38    40
Name: count, dtype: int64

In [15]:
matches.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf              float64
ga              float64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh              float64
sot             float64
dist            float64
fk              float64
pk                int64
pkatt             int64
season            int64
team             object
dtype: object

In [16]:
matches["date"] = pd.to_datetime(matches["date"])

In [17]:
matches

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2.0,1.0,Nott'ham Forest,...,Match Report,NaN,15.0,7.0,19.1,0.0,0,0,2024,Arsenal
1,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,Match Report,NaN,13.0,2.0,16.4,0.0,1,1,2024,Arsenal
2,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,Match Report,NaN,18.0,9.0,13.8,0.0,1,1,2024,Arsenal
3,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3.0,1.0,Manchester Utd,...,Match Report,NaN,17.0,5.0,15.0,0.0,0,0,2024,Arsenal
4,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1.0,0.0,Everton,...,Match Report,NaN,13.0,4.0,17.4,0.0,0,0,2024,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2409,2024-09-21,15:00,Premier League,Matchweek 5,Sat,Home,D,1.0,1.0,Ipswich Town,...,Match Report,NaN,11.0,3.0,13.7,0.0,0,0,2025,Southampton
2410,2024-09-30,20:00,Premier League,Matchweek 6,Mon,Away,L,1.0,3.0,Bournemouth,...,Match Report,NaN,9.0,3.0,18.4,0.0,0,0,2025,Southampton
2411,2024-10-05,15:00,Premier League,Matchweek 7,Sat,Away,L,1.0,3.0,Arsenal,...,Match Report,NaN,8.0,2.0,16.4,0.0,0,0,2025,Southampton
2412,2024-10-19,15:00,Premier League,Matchweek 8,Sat,Home,L,2.0,3.0,Leicester City,...,Match Report,NaN,14.0,7.0,9.6,0.0,0,0,2025,Southampton


In [18]:
matches.dtypes

date            datetime64[ns]
time                    object
comp                    object
round                   object
day                     object
venue                   object
result                  object
gf                     float64
ga                     float64
opponent                object
xg                     float64
xga                    float64
poss                   float64
attendance             float64
captain                 object
formation               object
referee                 object
match report            object
notes                  float64
sh                     float64
sot                    float64
dist                   float64
fk                     float64
pk                       int64
pkatt                    int64
season                   int64
team                    object
dtype: object

In [19]:
matches["venue_code"] = matches["venue"].astype("category").cat.codes #convert from string to categories, then categories into numbers

In [20]:
matches["opp_code"] = matches["opponent"].astype("category").cat.codes

In [21]:
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")

In [22]:
matches["day_code"] = matches["date"].dt.dayofweek

In [23]:
matches["target"] = (matches["result"] == "W").astype("int")

In [24]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,target
0,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2.0,1.0,Nott'ham Forest,...,0.0,0,0,2024,Arsenal,1,19,12,5,1
1,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,0.0,1,1,2024,Arsenal,0,7,20,0,1
2,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,0.0,1,1,2024,Arsenal,1,9,15,5,0
3,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3.0,1.0,Manchester Utd,...,0.0,0,0,2024,Arsenal,1,16,16,6,1
4,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1.0,0.0,Everton,...,0.0,0,0,2024,Arsenal,0,8,16,6,1


In [25]:
%pip install scikit-learn
from sklearn.ensemble import RandomForestClassifier

Note: you may need to restart the kernel to use updated packages.


In [26]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [27]:
train = matches[matches["date"] < '2024-10-25']

In [28]:
train.shape

(2394, 32)

In [29]:
test = matches[matches["date"] > '2024-10-25']
test

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,target
2242,2024-10-26,15:00,Premier League,Matchweek 9,Sat,Home,W,1.0,0.0,Southampton,...,0.0,0,0,2025,ManchesterCity,1,21,15,5,1
2251,2024-10-27,16:30,Premier League,Matchweek 9,Sun,Away,D,2.0,2.0,Arsenal,...,0.0,0,0,2025,Liverpool,0,0,16,6,0
2260,2024-10-27,16:30,Premier League,Matchweek 9,Sun,Home,D,2.0,2.0,Liverpool,...,0.0,0,0,2025,Arsenal,1,13,16,6,0
2269,2024-10-26,15:00,Premier League,Matchweek 9,Sat,Home,D,1.0,1.0,Bournemouth,...,0.0,0,0,2025,AstonVilla,1,2,15,5,0
2278,2024-10-27,14:00,Premier League,Matchweek 9,Sun,Home,W,2.0,1.0,Newcastle Utd,...,0.0,0,0,2025,Chelsea,1,17,14,6,1
2287,2024-10-26,15:00,Premier League,Matchweek 9,Sat,Home,D,2.0,2.0,Wolves,...,1.0,0,0,2025,BrightonandHoveAlbion,1,25,15,5,0
2305,2024-10-27,14:00,Premier League,Matchweek 9,Sun,Away,L,0.0,1.0,Crystal Palace,...,0.0,0,0,2025,TottenhamHotspur,0,7,14,6,0
2314,2024-10-26,15:00,Premier League,Matchweek 9,Sat,Home,W,4.0,3.0,Ipswich Town,...,1.0,1,1,2025,Brentford,1,10,15,5,1
2323,2024-10-26,17:30,Premier League,Matchweek 9,Sat,Away,D,1.0,1.0,Everton,...,0.0,0,0,2025,Fulham,0,8,17,5,0
2332,2024-10-26,15:00,Premier League,Matchweek 9,Sat,Away,D,1.0,1.0,Aston Villa,...,0.0,0,0,2025,Bournemouth,0,1,15,5,0


In [30]:
test.shape

(18, 32)

In [201]:
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [202]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [203]:
preds= rf.predict(test[predictors])
preds

array([1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [204]:
from sklearn.metrics import accuracy_score 

In [205]:
acc = accuracy_score(test["target"], preds)

In [206]:
acc

0.6111111111111112

In [207]:
combined = pd.DataFrame(dict(actual=test["target"], prediction = preds))

In [208]:
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1
actual,,
0,10,3
1,4,1


In [209]:
from sklearn.metrics import precision_score 

In [210]:
precision_score(test["target"], preds)
#our precision was only 48%, when we predicted a win, the team only actually won 47% of the time

np.float64(0.25)

In [211]:
#create more predictors, to improve accuracy of our model. Lets try and look at teams current form

#lets create a df for every squad in our data
grouped_matches = matches.groupby("team")

In [212]:
group = grouped_matches.get_group("Arsenal")
group

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,target
0,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2.0,1.0,Nott'ham Forest,...,0.0,0,0,2024,Arsenal,1,19,12,5,1
1,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,0.0,1,1,2024,Arsenal,0,7,20,0,1
2,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,0.0,1,1,2024,Arsenal,1,9,15,5,0
3,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3.0,1.0,Manchester Utd,...,0.0,0,0,2024,Arsenal,1,16,16,6,1
4,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1.0,0.0,Everton,...,0.0,0,0,2024,Arsenal,0,8,16,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256,2024-09-22,16:30,Premier League,Matchweek 5,Sun,Away,D,2.0,2.0,Manchester City,...,0.0,0,0,2025,Arsenal,0,15,16,6,0
2257,2024-09-28,15:00,Premier League,Matchweek 6,Sat,Home,W,4.0,2.0,Leicester City,...,0.0,0,0,2025,Arsenal,1,12,15,5,1
2258,2024-10-05,15:00,Premier League,Matchweek 7,Sat,Home,W,3.0,1.0,Southampton,...,0.0,0,0,2025,Arsenal,1,21,15,5,1
2259,2024-10-19,17:30,Premier League,Matchweek 8,Sat,Away,L,0.0,2.0,Bournemouth,...,0.0,0,0,2025,Arsenal,0,2,17,5,0


In [213]:
def rolling_averages(group, cols, new_cols):
    group= group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed="left").mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group


In [214]:
#cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
cols = ["gf", "ga", "sh", "sot"]
new_cols = [f"{c}_rolling" for c in cols]

In [215]:
new_cols #new columsn we want to create with the rolling avgs

['gf_rolling', 'ga_rolling', 'sh_rolling', 'sot_rolling']

In [216]:
rolling_averages(group, cols, new_cols) # for Arsenal

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,team,venue_code,opp_code,hour,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling
1629,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Home,W,1.0,0.0,Norwich City,...,Arsenal,1,18,15,5,1,0.000000,3.000000,9.666667,2.333333
1630,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Away,W,1.0,0.0,Burnley,...,Arsenal,0,5,15,5,1,0.333333,2.333333,12.333333,3.000000
1631,2021-09-26,16:30,Premier League,Matchweek 6,Sun,Home,W,3.0,1.0,Tottenham,...,Arsenal,1,22,16,6,1,0.666667,1.666667,14.666667,3.000000
1632,2021-10-02,17:30,Premier League,Matchweek 7,Sat,Away,D,0.0,0.0,Brighton,...,Arsenal,0,4,17,5,0,1.666667,0.333333,18.333333,5.333333
1633,2021-10-18,20:00,Premier League,Matchweek 8,Mon,Home,D,2.0,2.0,Crystal Palace,...,Arsenal,1,7,20,0,0,1.333333,0.333333,11.000000,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256,2024-09-22,16:30,Premier League,Matchweek 5,Sun,Away,D,2.0,2.0,Manchester City,...,Arsenal,0,15,16,6,0,1.333333,0.333333,9.000000,5.000000
2257,2024-09-28,15:00,Premier League,Matchweek 6,Sat,Home,W,4.0,2.0,Leicester City,...,Arsenal,1,12,15,5,1,1.333333,1.000000,7.666667,4.666667
2258,2024-10-05,15:00,Premier League,Matchweek 7,Sat,Home,W,3.0,1.0,Southampton,...,Arsenal,1,21,15,5,1,2.333333,1.333333,15.666667,7.666667
2259,2024-10-19,17:30,Premier League,Matchweek 8,Sat,Away,L,0.0,2.0,Bournemouth,...,Arsenal,0,2,17,5,0,3.000000,1.666667,23.000000,8.333333


In [217]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

C:\Users\tomas\AppData\Local\Temp\ipykernel_18996\4052147919.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))


In [218]:
matches_rolling

date   time            comp        round  \
team                                                                         
Arsenal                1629 2021-09-11  15:00  Premier League  Matchweek 4   
                       1630 2021-09-18  15:00  Premier League  Matchweek 5   
                       1631 2021-09-26  16:30  Premier League  Matchweek 6   
                       1632 2021-10-02  17:30  Premier League  Matchweek 7   
                       1633 2021-10-18  20:00  Premier League  Matchweek 8   
...                                ...    ...             ...          ...   
WolverhamptonWanderers 2400 2024-09-21  15:00  Premier League  Matchweek 5   
                       2401 2024-09-28  17:30  Premier League  Matchweek 6   
                       2402 2024-10-05  15:00  Premier League  Matchweek 7   
                       2403 2024-10-20  14:00  Premier League  Matchweek 8   
                       2404 2024-10-26  15:00  Premier League  Matchweek 9   

                             day venue result   gf   ga         opponent  ...  \
team                                                                      ...   
Arsenal                1629  Sat  Home      W  1.0  0.0     Norwich City  ...   
                       1630  Sat  Away      W  1.0  0.0          Burnley  ...   
                       1631  Sun  Home      W  3.0  1.0        Tottenham  ...   
                       1632  Sat  Away      D  0.0  0.0         Brighton  ...   
                       1633  Mon  Home      D  2.0  2.0   Crystal Palace  ...   
...                          ...   ...    ...  ...  ...              ...  ...   
WolverhamptonWanderers 2400  Sat  Away      L  1.0  3.0      Aston Villa  ...   
                       2401  Sat  Home      L  1.0  2.0        Liverpool  ...   
                       2402  Sat  Away      L  3.0  5.0        Brentford  ...   
                       2403  Sun  Home      L  1.0  2.0  Manchester City  ...   
                       2404  Sat  Away      D  2.0  2.0         Brighton  ...   

                                               team  venue_code  opp_code  \
team                                                                        
Arsenal                1629                 Arsenal           1        18   
                       1630                 Arsenal           0         5   
                       1631                 Arsenal           1        22   
                       1632                 Arsenal           0         4   
                       1633                 Arsenal           1         7   
...                                             ...         ...       ...   
WolverhamptonWanderers 2400  WolverhamptonWanderers           0         1   
                       2401  WolverhamptonWanderers           1        13   
                       2402  WolverhamptonWanderers           0         3   
                       2403  WolverhamptonWanderers           1        15   
                       2404  WolverhamptonWanderers           0         4   

                             hour day_code target gf_rolling ga_rolling  \
team                                                                      
Arsenal                1629    15        5      1   0.000000   3.000000   
                       1630    15        5      1   0.333333   2.333333   
                       1631    16        6      1   0.666667   1.666667   
                       1632    17        5      0   1.666667   0.333333   
                       1633    20        0      0   1.333333   0.333333   
...                           ...      ...    ...        ...        ...   
WolverhamptonWanderers 2400    15        5      0   1.333333   3.000000   
                       2401    17        5      0   1.000000   2.000000   
                       2402    15        5      0   1.000000   2.333333   
                       2403    14        6      0   1.666667   3.333333   
                       2404    15        5      0   1.666667   

In [219]:
matches_rolling = matches_rolling.droplevel('team')

In [220]:
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,team,venue_code,opp_code,hour,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling
1629,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Home,W,1.0,0.0,Norwich City,...,Arsenal,1,18,15,5,1,0.000000,3.000000,9.666667,2.333333
1630,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Away,W,1.0,0.0,Burnley,...,Arsenal,0,5,15,5,1,0.333333,2.333333,12.333333,3.000000
1631,2021-09-26,16:30,Premier League,Matchweek 6,Sun,Home,W,3.0,1.0,Tottenham,...,Arsenal,1,22,16,6,1,0.666667,1.666667,14.666667,3.000000
1632,2021-10-02,17:30,Premier League,Matchweek 7,Sat,Away,D,0.0,0.0,Brighton,...,Arsenal,0,4,17,5,0,1.666667,0.333333,18.333333,5.333333
1633,2021-10-18,20:00,Premier League,Matchweek 8,Mon,Home,D,2.0,2.0,Crystal Palace,...,Arsenal,1,7,20,0,0,1.333333,0.333333,11.000000,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2400,2024-09-21,15:00,Premier League,Matchweek 5,Sat,Away,L,1.0,3.0,Aston Villa,...,WolverhamptonWanderers,0,1,15,5,0,1.333333,3.000000,11.666667,3.666667
2401,2024-09-28,17:30,Premier League,Matchweek 6,Sat,Home,L,1.0,2.0,Liverpool,...,WolverhamptonWanderers,1,13,17,5,0,1.000000,2.000000,11.000000,3.666667
2402,2024-10-05,15:00,Premier League,Matchweek 7,Sat,Away,L,3.0,5.0,Brentford,...,WolverhamptonWanderers,0,3,15,5,0,1.000000,2.333333,10.000000,4.000000
2403,2024-10-20,14:00,Premier League,Matchweek 8,Sun,Home,L,1.0,2.0,Manchester City,...,WolverhamptonWanderers,1,15,14,6,0,1.666667,3.333333,11.666667,4.333333


In [221]:
matches_rolling.index = range(matches_rolling.shape[0]) #reset the indices

In [222]:
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,team,venue_code,opp_code,hour,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling
0,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Home,W,1.0,0.0,Norwich City,...,Arsenal,1,18,15,5,1,0.000000,3.000000,9.666667,2.333333
1,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Away,W,1.0,0.0,Burnley,...,Arsenal,0,5,15,5,1,0.333333,2.333333,12.333333,3.000000
2,2021-09-26,16:30,Premier League,Matchweek 6,Sun,Home,W,3.0,1.0,Tottenham,...,Arsenal,1,22,16,6,1,0.666667,1.666667,14.666667,3.000000
3,2021-10-02,17:30,Premier League,Matchweek 7,Sat,Away,D,0.0,0.0,Brighton,...,Arsenal,0,4,17,5,0,1.666667,0.333333,18.333333,5.333333
4,2021-10-18,20:00,Premier League,Matchweek 8,Mon,Home,D,2.0,2.0,Crystal Palace,...,Arsenal,1,7,20,0,0,1.333333,0.333333,11.000000,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2331,2024-09-21,15:00,Premier League,Matchweek 5,Sat,Away,L,1.0,3.0,Aston Villa,...,WolverhamptonWanderers,0,1,15,5,0,1.333333,3.000000,11.666667,3.666667
2332,2024-09-28,17:30,Premier League,Matchweek 6,Sat,Home,L,1.0,2.0,Liverpool,...,WolverhamptonWanderers,1,13,17,5,0,1.000000,2.000000,11.000000,3.666667
2333,2024-10-05,15:00,Premier League,Matchweek 7,Sat,Away,L,3.0,5.0,Brentford,...,WolverhamptonWanderers,0,3,15,5,0,1.000000,2.333333,10.000000,4.000000
2334,2024-10-20,14:00,Premier League,Matchweek 8,Sun,Home,L,1.0,2.0,Manchester City,...,WolverhamptonWanderers,1,15,14,6,0,1.666667,3.333333,11.666667,4.333333


In [230]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2024-10-25']
    test = data[data["date"] > '2024-10-25']
    rf.fit(train[predictors], train["target"])
    preds= rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual = test["target"], predicted = preds), index=test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision

In [231]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [232]:
precision

np.float64(0.42857142857142855)

In [233]:
combined

,actual,predicted
117,0,1
235,0,0
315,0,1
433,1,0
550,0,1
738,1,1
856,1,0
974,0,0
1054,0,0
1060,0,0


In [234]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)

In [235]:
combined

,actual,predicted,date,team,opponent,result
117,0,1,2024-10-27,Arsenal,Liverpool,D
235,0,0,2024-10-26,AstonVilla,Bournemouth,D
315,0,1,2024-10-26,Bournemouth,Aston Villa,D
433,1,0,2024-10-26,Brentford,Ipswich Town,W
550,0,1,2024-10-26,BrightonandHoveAlbion,Wolves,D
738,1,1,2024-10-27,Chelsea,Newcastle Utd,W
856,1,0,2024-10-27,CrystalPalace,Tottenham,W
974,0,0,2024-10-26,Everton,Fulham,D
1054,0,0,2024-10-26,Fulham,Everton,D
1060,0,0,2024-10-26,IpswichTown,Brentford,L


In [261]:
standings_url = 'https://fbref.com/en/comps/9/Premier-League-Stats'

In [262]:
all_matches = []

In [ ]:
import time
import requests
from bs4 import BeautifulSoup


data = requests.get(standings_url)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0] #css selector to select table with class name stats_table

links = [l.get('href') for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]  #only want the links with /squads/ in it
team_urls = [f"https://fbref.com{l}" for l in links] #create list of all the teams urls/links


for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", "")

    data = requests.get(team_url)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
    time.sleep(5)

    soup = BeautifulSoup(data.text)
    links = [l.get('href') for l in soup.find_all("a")]
    links = [l for l in links if l and 'all_comps/shooting/' in l] #get link to shooting stats for team 
    link = f"https://fbref.com{links[0]}"
    data = requests.get(link)
    shooting = pd.read_html(data.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()

    try:
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
    except ValueError:
        continue
    
    team_data = matches[matches["Comp"] == "Premier League"]
    print(team_name)
    team_data["Season"] = 2025 #adding season column
    team_data["Team"] = team_name #adding team collumn
    all_matches.append(team_data)
    time.sleep(1)

C:\Users\tomas\AppData\Local\Temp\ipykernel_18996\1211466457.py:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\tomas\AppData\Local\Temp\ipykernel_18996\1211466457.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\tomas\AppData\Local\Temp\ipykernel_18996\1211466457.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_data["Season"] = 2025 #adding season co

Liverpool


C:\Users\tomas\AppData\Local\Temp\ipykernel_18996\1211466457.py:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]


In [270]:
match_df = pd.concat(all_matches)

In [271]:
match_df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Poss,Attendance,Captain,Formation,Opp Formation,Referee,Match Report,Notes,Season,Team
0,2024-08-17,12:30,Premier League,Matchweek 1,Sat,Away,W,2.0,0.0,Ipswich Town,...,62.0,30014.0,Virgil van Dijk,4-2-3-1,4-2-3-1,Tim Robinson,Match Report,NaN,2025,Liverpool
1,2024-08-25,16:30,Premier League,Matchweek 2,Sun,Home,W,2.0,0.0,Brentford,...,62.0,60017.0,Virgil van Dijk,4-2-3-1,4-4-2,Stuart Attwell,Match Report,NaN,2025,Liverpool
2,2024-09-01,16:00,Premier League,Matchweek 3,Sun,Away,W,3.0,0.0,Manchester Utd,...,47.0,73738.0,Virgil van Dijk,4-2-3-1,4-2-3-1,Anthony Taylor,Match Report,NaN,2025,Liverpool
3,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,L,0.0,1.0,Nott'ham Forest,...,68.0,60344.0,Virgil van Dijk,4-2-3-1,4-2-3-1,Michael Oliver,Match Report,NaN,2025,Liverpool
5,2024-09-21,15:00,Premier League,Matchweek 5,Sat,Home,W,3.0,0.0,Bournemouth,...,58.0,60347.0,Virgil van Dijk,4-2-3-1,4-2-3-1,Tony Harrington,Match Report,NaN,2025,Liverpool
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2025-04-26,15:00,Premier League,Matchweek 34,Sat,Home,NaN,NaN,NaN,Leicester City,...,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2025,WolverhamptonWanderers
36,2025-05-03,15:00,Premier League,Matchweek 35,Sat,Away,NaN,NaN,NaN,Manchester City,...,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2025,WolverhamptonWanderers
37,2025-05-10,15:00,Premier League,Matchweek 36,Sat,Home,NaN,NaN,NaN,Brighton,...,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2025,WolverhamptonWanderers
38,2025-05-18,15:00,Premier League,Matchweek 37,Sun,Away,NaN,NaN,NaN,Crystal Palace,...,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2025,WolverhamptonWanderers


In [281]:
match_df.columns

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'opp formation', 'referee', 'match report', 'notes', 'season', 'team'],
      dtype='object')

In [275]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,poss,attendance,captain,formation,opp formation,referee,match report,notes,season,team
0,2024-11-09,20:00,Premier League,Matchweek 11,Sat,Home,NaN,NaN,NaN,Aston Villa,...,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2025,Liverpool
1,2024-11-10,14:00,Premier League,Matchweek 11,Sun,Home,NaN,NaN,NaN,Newcastle Utd,...,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2025,NottinghamForest
2,2024-11-10,16:30,Premier League,Matchweek 11,Sun,Home,NaN,NaN,NaN,Arsenal,...,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2025,Chelsea
3,2024-11-10,14:00,Premier League,Matchweek 11,Sun,Home,NaN,NaN,NaN,Ipswich Town,...,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2025,TottenhamHotspur
4,2024-11-09,17:30,Premier League,Matchweek 11,Sat,Home,NaN,NaN,NaN,Manchester City,...,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2025,BrightonandHoveAlbion
5,2024-11-09,15:00,Premier League,Matchweek 11,Sat,Home,NaN,NaN,NaN,Bournemouth,...,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2025,Brentford
6,2024-11-10,14:00,Premier League,Matchweek 11,Sun,Home,NaN,NaN,NaN,Leicester City,...,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2025,ManchesterUnited
7,2024-11-09,15:00,Premier League,Matchweek 11,Sat,Home,NaN,NaN,NaN,Everton,...,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2025,WestHamUnited
8,2024-11-09,15:00,Premier League,Matchweek 11,Sat,Home,NaN,NaN,NaN,Fulham,...,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2025,CrystalPalace
9,2024-11-09,15:00,Premier League,Matchweek 11,Sat,Home,NaN,NaN,NaN,Southampton,...,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2025,WolverhamptonWanderers


In [276]:
match_df.shape

(10, 22)

In [279]:
def make_predictions(dataTrain, dataTest, predictors):
    train = dataTrain[dataTrain["date"] < '2024-11-06']
    test = dataTest
    rf.fit(train[predictors], train["target"])
    preds= rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual = test["target"], predicted = preds), index=test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision

In [280]:
combined, precision = make_predictions(matches_rolling, match_df, predictors + new_cols)

KeyError: "None of [Index(['venue_code', 'opp_code', 'hour', 'day_code', 'gf_rolling',\n       'ga_rolling', 'sh_rolling', 'sot_rolling'],\n      dtype='object')] are in the [columns]"